In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
URM_test=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_test_leaderboard.npz")

In [ ]:
URM_test.shape

(4439758, 23692)

In [ ]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [ ]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [ ]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [ ]:
is_sorted(Session_ids)

True

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/leaderboard/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
15952526
loaded UCF
0
loading ICBF
14870776
loaded ICBF
0
loading Graph
8315730
loaded Graph
0
loading ICF
17772188
loaded ICF


In [ ]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids

In [ ]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/leaderboard/candidates.csv")
# submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/leaderboard/candidates.csv")
# submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/leaderboard/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# del submission_df
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
# del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [ ]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [ ]:
dataset.fillna(0,inplace=True)

In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_leaderboard.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items'])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [ ]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [ ]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Session_Id,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,...,Max_Score_ICF,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,21,0.100227,0.0,0.019740,0.000000,26,0.000000,0.0,55.290478,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,52,0.242711,0.0,0.087397,0.548803,26,0.002269,33.0,55.290478,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,80,0.090264,0.0,0.045723,0.545978,26,0.001017,76.0,55.290478,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,115,0.025982,0.0,0.000000,0.000000,26,0.000000,0.0,55.290478,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,159,0.058162,0.0,0.020845,0.000000,26,0.000000,0.0,55.290478,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
gc.collect()

2288

In [ ]:
dataset=dataset.sort_values("Session_Id")

In [ ]:
gc.collect()

0

In [ ]:
import lightgbm

In [ ]:
k=10

In [ ]:
result = []

In [ ]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'./drive/MyDrive/Recommendation_system/rankers/lightgbm/all_traditional_and_lstm/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))
    # dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))

#combine scores
result = np.array(result)
result

array([[-1.06199717, -1.28723011, -0.75361234, ..., -1.84409324,
        -1.74050445, -1.89948004],
       [-1.90762996, -2.04852597, -1.1294881 , ..., -3.11491342,
        -2.85366148, -3.14014392],
       [-2.03907294, -2.22644131, -1.39615621, ..., -3.09806714,
        -2.85062155, -2.94957414],
       ...,
       [-2.14556097, -2.34619512, -1.60974023, ..., -3.33354656,
        -2.97198005, -3.02192806],
       [-2.0443138 , -2.41589775, -1.28545983, ..., -3.49702238,
        -3.06426512, -3.00437757],
       [-2.13352195, -2.32995087, -1.39751576, ..., -3.25758569,
        -2.93342442, -3.18958867]])

In [ ]:
result.shape

(10, 30090188)

In [ ]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [ ]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

0

In [ ]:
dataset.head()

,Session_Id,Item_ID,score
0,26,21,-1.792160
15952582,26,1483,-2.016757
15952583,26,1524,-1.178805
15952584,26,1607,-1.945961
15952585,26,1643,-1.240325


In [ ]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [ ]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [ ]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [ ]:
dataset.iloc[0]

Session_Id                                                   26
Item_ID       [21, 1483, 1524, 1607, 1643, 1649, 1651, 1464,...
score         [-1.7921601232390505, -2.016757311270276, -1.1...
indices       [369, 400, 85, 108, 491, 302, 77, 289, 83, 98,...
Name: 0, dtype: object

In [ ]:
dataset.iloc[0]["Item_ID"][536]

3338

In [ ]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [ ]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [ ]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [ ]:
dataset

,Session_Id,predictions,rank
0,26,"[606, 1008, 4881, 4952, 3646, 2905, 4685, 3894...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[772, 1827, 270, 3400, 3229, 4878, 3656, 3676,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[2289, 3398, 1181, 4986, 2235, 3203, 588, 3355...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[2378, 4900, 3337, 534, 1633, 2369, 3937, 3922...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[1004, 3172, 2175, 25, 1518, 3022, 3586, 1768,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[3526, 4810, 2258, 2605, 1468, 3084, 2800, 949...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[1875, 2877, 3880, 3155, 867, 3153, 3026, 2919...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[1901, 4846, 4779, 2638, 625, 3487, 2431, 2835...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[2596, 1015, 42, 2266, 3531, 2638, 36, 360, 36...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [ ]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [ ]:
dataset.columns=["session_id","code","rank"]

In [ ]:
dataset.head()

,session_id,code,rank
0,26,"(606, 1008, 4881, 4952, 3646, 2905, 4685, 3894...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(772, 1827, 270, 3400, 3229, 4878, 3656, 3676,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(2289, 3398, 1181, 4986, 2235, 3203, 588, 3355...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(2378, 4900, 3337, 534, 1633, 2369, 3937, 3922...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(1004, 3172, 2175, 25, 1518, 3022, 3586, 1768,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [ ]:
mapping= pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [ ]:
print(len(submission_df))

5000000


In [ ]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [ ]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [ ]:
submission_df.to_csv(
    f'./drive/MyDrive/Recommendation_system/rankers/lightgbm/all_traditional_and_lstm/leaderboard/leaderboard_sub.csv',
    index=False
    )